### Reconstructing the path

#### Task description
In this assignment you will use Spark to compute the shortest path between two vertices. In the video, you have learned how to compute the distances between a source vertex and all other vertices in a graph. Now, your task is to reconstruct the shortest path, that is a sequence of vertices connected by the edges.

Dataset location: /data/twitter/twitter_sample.txt

Format: user_id < tab > follower_id

In [1]:
from __future__ import print_function
from pyspark import SparkConf, SparkContext

In [2]:
sc = SparkContext(conf=SparkConf().setAppName("reconstructing-the-path").setMaster("local"))

In [3]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

def step(item):
    curr, verts, flag, next_key = item[0], item[1][0][0], item[1][0][1], item[1][1]
    flag = next_key == 34
    verts = list(verts)
    verts.append(curr)
    if flag:
        verts.append(next_key)
    return (next_key, (verts, flag))

In [4]:
n = 10  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample.txt").map(parse_edge).map(lambda e: (e[1], e[0])).partitionBy(n).cache()
x = 12
distances = sc.parallelize([(x, (list(), False))]).partitionBy(n)

ittr = 1

while True:
    candidates = distances.join(edges, n).map(step).persist()
    count = candidates.filter(lambda r: r[1][1]).count()
    
    count_all = candidates.count()
    
    
    print('itteration %d, all count %d, count %d' % (ittr, count_all, count))
    print('-------------------------------------------------------------------------------------------')
    
    if count_all == 0:
        break
    
    if count > 0:
        break
    else:
        distances = candidates 
        ittr += 1

itteration 1, all count 4, count 0
-------------------------------------------------------------------------------------------
itteration 2, all count 4, count 0
-------------------------------------------------------------------------------------------
itteration 3, all count 15, count 0
-------------------------------------------------------------------------------------------
itteration 4, all count 31, count 0
-------------------------------------------------------------------------------------------
itteration 5, all count 89, count 0
-------------------------------------------------------------------------------------------
itteration 6, all count 201, count 0
-------------------------------------------------------------------------------------------
itteration 7, all count 518, count 0
-------------------------------------------------------------------------------------------
itteration 8, all count 1173, count 1
------------------------------------------------------------------

In [5]:
tmp = candidates.filter(lambda r: r[1][1]).map(lambda r: (r[0], ','.join([str(i) for i in r[1][0]])))

In [6]:
for row in tmp.take(1): print(row[1])

12,422,53,52,107,20,23,274,34
